## Automatyczne Uczenie Maszynowe
#### Laboratorium 4

*AutoGluon. Optymalizacja hipermarametór.*

### Zadanie 1
-----
Wykorzystując bazę zbiorów danych [OpenML](https://www.openml.org/) wybierz 3 zbiory (najlepiej z brakami danych):

a) zbiór do regresji

b) zbiór do klasyfikacji 

c) zbiór do multiklasyfikacji

Każdy zbiór danych podziel na część treningową i testową. Używając pakietu `AutoGluon` przygotuj modele i sprawdź ich jakość zarówno na zbiorze treningowym i testowym.

Odpowiedz na pytania:

1. Jakie są domyślne parametry i jakie wartości przyjmują?

2. Czy możemy określić czas działania pakietu? Jeżeli tak, to przetestuj 2-3 różne czasy i przedstaw wyniki modeli.

3. Jaki sposób optymalizacji hiperparametrów jest dostępny w `AutoGluon`?

4. Czy wyniki dla zespołów modeli (ensemblingi) są lepsze od poszczególnych modeli?

### Zadanie 2
----
Dla wybranego zbioru danych z Zadanie 1 przygotuj 2-3 modele na których zastosuj optymalizację hiperparametrów używając metody *random search*.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Porównaj uzyskane wyniki z wynikami Zadania 1. Które podejście otrzymuje lepsze wyniki? AutoGluon czy RandomSearch?

### Zadanie 3
----
Wykorzystując pakiet `scikit-optimize` przygotuj optymalizację hiperparametrów metodą bayesowską. 

https://scikit-optimize.github.io/stable/

https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/master/bayesian_optimization.ipynb#scrollTo=sJMlow8JPDoi

Porównaj uzyskane wyniki z wynikami Zadania 1 oraz Zadania 2.